# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [1]:
!nvidia-smi

Wed Jun 21 23:29:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:3F:00.0 Off |                    0 |
| N/A   28C    P0    41W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:40:00.0 Off |                    0 |
| N/A   

# Download data
If the Google Drive links below do not work, you can use the dropbox link below or download data from [Kaggle](https://www.kaggle.com/t/a339b77fa5214978bfb8dde62d3151fe), and upload data manually to the workspace.

In [2]:
# google drive link
# !pip install gdown
# !gdown --id '1BjXalPZxq9mybPKNjF3h5L3NcF7XKTS-' --output covid_train.csv
# !gdown --id '1B55t74Jg2E5FCsKCsUEkPKIuqaY7UIi1' --output covid_test.csv

# dropbox link
!wget -O covid_train.csv https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
!wget -O covid_test.csv https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/root/.wget-hsts'. HSTS will be disabled.
--2023-06-21 23:29:54--  https://www.dropbox.com/s/lmy1riadzoy0ahw/covid.train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 104.244.43.208, 2a03:2880:f126:83:face:b00c:0:25de
Connecting to www.dropbox.com (www.dropbox.com)|104.244.43.208|:443... failed: Connection timed out.
Connecting to www.dropbox.com (www.dropbox.com)|2a03:2880:f126:83:face:b00c:0:25de|:443... failed: Address family not supported by protocol.
Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/root/.wget-hsts'. HSTS will be disabled.
--2023-06-21 23:29:57--  https://www.dropbox.com/s/zalbw42lu4nmhr2/covid.test.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 75.126.135.131, 2a03:2880:f134:83:face:b00c:0:25de
Connecting to www.dropbox.com (www.dropbox.com)|75.126.13

# Import packages

In [3]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [4]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():  #                  
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [5]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [30]:
# class My_Model(nn.Module):
#     def __init__(self, input_dim):
#         super(My_Model, self).__init__()
#         # TODO: modify model's structure, be aware of dimensions. 
#         self.layers = nn.Sequential(
#             nn.Linear(input_dim, 16),
#             nn.ReLU(),
#             nn.Linear(16, 8),
#             nn.ReLU(),
#             nn.Linear(8, 1)
#         )

#     def forward(self, x):
#         x = self.layers(x)
#         x = x.squeeze(1) # (B, 1) -> (B)
#         return x

class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(32),
            nn.Dropout(0.1),

            nn.Linear(32, 16),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(16),
            nn.Dropout(0.1),

            nn.Linear(16, 8),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(8),
            nn.Dropout(0.1),

            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [74]:
# def select_feat(train_data, valid_data, test_data, select_all=True):
#     '''Selects useful features to perform regression'''
#     y_train, y_valid = train_data[:,-1], valid_data[:,-1]
#     raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

#     if select_all:
#         feat_idx = list(range(raw_x_train.shape[1]))
#     else:
#         feat_idx = list(range(35, raw_x_train.shape[1])) # TODO: Select suitable feature columns.
        
#     return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

features = pd.read_csv('./covid_train.csv')
x_data, y_data = features.iloc[:,:-1], features.iloc[:,-1]
k = 24
selector = SelectKBest(score_func=f_regression, k=k)
result = selector.fit(x_data, y_data)
idx = np.argsort(result.scores_)[::-1]
selected_idx = list(np.sort(idx[:k]))

# Training Loop

In [92]:
# def getLoss(pred, target, model):
#     loss = nn.MSELoss(reduction='mean')
#     ''' Calculate loss '''
#     regularization_loss = 0
#     for param in model.parameters():
#         # 使用L2正则项
#         regularization_loss += torch.sum(param ** 2)
#     return loss(pred, target) + 0.00075 * regularization_loss

def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    
    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
#     optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7) 
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4) 
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=2, T_mult=2, eta_min=1e-5)
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
#         train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
#             train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#             train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)
        
        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session. best _loss is ', best_loss)
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [93]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# config = {
#     'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
#     'select_all': True,   # Whether to use all features.
#     'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
#     'n_epochs': 5000,     # Number of epochs.            
#     'batch_size': 256, 
#     'learning_rate': 1e-5,              
#     'early_stop': 600,    # If model has not improved for this many consecutive epochs, stop training.     
#     'save_path': './models/model.ckpt'  # Your model will be saved here.
# }
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': False,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 5000,     # Number of epochs.            
    'batch_size': 512, 
    'learning_rate': 1e-5,              
    'early_stop': 2000,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [94]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 3009 x 89 (35 states + 18 features x 3 days) 
# test_data size: 997 x 88 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid_train.csv').values, pd.read_csv('./covid_test.csv').values
print('test_data ', pd.read_csv('./covid_train.csv').columns.tolist())
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])
x_mean, x_std = x_train.mean(axis=0), x_train.std(axis=0)
x_train = (x_train - x_mean) / x_std
x_valid = (x_valid - x_mean) / x_std
x_test = (x_test - x_mean) / x_std
# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

test_data  ['id', 'AL', 'AZ', 'CA', 'CO', 'CT', 'FL', 'GA', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MO', 'NJ', 'NM', 'NY', 'NC', 'OH', 'OK', 'OR', 'PA', 'SC', 'TN', 'TX', 'VA', 'WA', 'WV', 'WI', 'cli', 'ili', 'wnohh_cmnty_cli', 'wbelief_masking_effective', 'wbelief_distancing_effective', 'wcovid_vaccinated_friends', 'wlarge_event_indoors', 'wothers_masked_public', 'wothers_distanced_public', 'wshop_indoors', 'wrestaurant_indoors', 'wworried_catch_covid', 'hh_cmnty_cli', 'nohh_cmnty_cli', 'wearing_mask_7d', 'public_transit', 'worried_finances', 'tested_positive', 'cli.1', 'ili.1', 'wnohh_cmnty_cli.1', 'wbelief_masking_effective.1', 'wbelief_distancing_effective.1', 'wcovid_vaccinated_friends.1', 'wlarge_event_indoors.1', 'wothers_masked_public.1', 'wothers_distanced_public.1', 'wshop_indoors.1', 'wrestaurant_indoors.1', 'wworried_catch_covid.1', 'hh_cmnty_cli.1', 'nohh_cmnty_cli.1', 'wearing_mask_7d.1', 'public_transit.1', 'worried_finances.1', 'tested_positiv

# Start training!

In [95]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/5000]: Train loss: 397.4158, Valid loss: 431.4609
Saving model with loss 431.461...
Epoch [2/5000]: Train loss: 383.1488, Valid loss: 412.1874
Saving model with loss 412.187...
Epoch [3/5000]: Train loss: 375.1366, Valid loss: 371.6339
Saving model with loss 371.634...
Epoch [4/5000]: Train loss: 366.7350, Valid loss: 388.3000
Epoch [5/5000]: Train loss: 357.9926, Valid loss: 353.8301
Saving model with loss 353.830...
Epoch [6/5000]: Train loss: 349.7455, Valid loss: 364.4962
Epoch [7/5000]: Train loss: 341.1416, Valid loss: 357.5255
Epoch [8/5000]: Train loss: 332.5958, Valid loss: 335.2576
Saving model with loss 335.258...
Epoch [9/5000]: Train loss: 323.1261, Valid loss: 329.1396
Saving model with loss 329.140...
Epoch [10/5000]: Train loss: 310.1986, Valid loss: 309.6425
Saving model with loss 309.643...
Epoch [11/5000]: Train loss: 300.9946, Valid loss: 308.3568
Saving model with loss 308.357...
Epoch [12/5000]: Train loss: 290.2818, Valid loss: 294.9649
Saving model with

Epoch [112/5000]: Train loss: 7.6711, Valid loss: 1.5259
Epoch [113/5000]: Train loss: 7.2136, Valid loss: 1.1107
Epoch [114/5000]: Train loss: 7.4778, Valid loss: 1.2316
Epoch [115/5000]: Train loss: 7.6250, Valid loss: 1.2634
Epoch [116/5000]: Train loss: 7.2078, Valid loss: 1.3971
Epoch [117/5000]: Train loss: 7.7334, Valid loss: 1.1535
Epoch [118/5000]: Train loss: 7.3748, Valid loss: 1.2579
Epoch [119/5000]: Train loss: 7.1890, Valid loss: 1.1764
Epoch [120/5000]: Train loss: 7.4345, Valid loss: 1.2755
Epoch [121/5000]: Train loss: 7.2839, Valid loss: 1.0778
Epoch [122/5000]: Train loss: 6.8384, Valid loss: 1.2204
Epoch [123/5000]: Train loss: 6.9962, Valid loss: 1.2668
Epoch [124/5000]: Train loss: 7.3756, Valid loss: 1.1599
Epoch [125/5000]: Train loss: 6.8342, Valid loss: 1.5860
Epoch [126/5000]: Train loss: 7.6353, Valid loss: 1.0998
Epoch [127/5000]: Train loss: 7.5221, Valid loss: 1.3223
Epoch [128/5000]: Train loss: 6.9805, Valid loss: 1.2155
Epoch [129/5000]: Train loss: 6

Epoch [259/5000]: Train loss: 6.4511, Valid loss: 1.1482
Epoch [260/5000]: Train loss: 6.4137, Valid loss: 1.0826
Epoch [261/5000]: Train loss: 7.2643, Valid loss: 1.2303
Epoch [262/5000]: Train loss: 6.7777, Valid loss: 1.1632
Epoch [263/5000]: Train loss: 6.7837, Valid loss: 0.9876
Epoch [264/5000]: Train loss: 6.4926, Valid loss: 1.0342
Epoch [265/5000]: Train loss: 7.1611, Valid loss: 1.1792
Epoch [266/5000]: Train loss: 6.7730, Valid loss: 1.1329
Epoch [267/5000]: Train loss: 6.4393, Valid loss: 1.0725
Epoch [268/5000]: Train loss: 6.4841, Valid loss: 1.1541
Epoch [269/5000]: Train loss: 6.4027, Valid loss: 1.0265
Epoch [270/5000]: Train loss: 6.4325, Valid loss: 1.0566
Epoch [271/5000]: Train loss: 6.2886, Valid loss: 1.1476
Epoch [272/5000]: Train loss: 6.4700, Valid loss: 0.9229
Epoch [273/5000]: Train loss: 7.0758, Valid loss: 1.1018
Epoch [274/5000]: Train loss: 6.5848, Valid loss: 1.1053
Epoch [275/5000]: Train loss: 6.9340, Valid loss: 1.3850
Epoch [276/5000]: Train loss: 6

Epoch [408/5000]: Train loss: 7.0004, Valid loss: 1.2680
Epoch [409/5000]: Train loss: 7.0866, Valid loss: 1.1147
Epoch [410/5000]: Train loss: 6.3885, Valid loss: 1.0899
Epoch [411/5000]: Train loss: 6.3275, Valid loss: 1.0491
Epoch [412/5000]: Train loss: 5.9987, Valid loss: 1.4169
Epoch [413/5000]: Train loss: 6.5286, Valid loss: 1.0590
Epoch [414/5000]: Train loss: 6.4259, Valid loss: 0.9863
Epoch [415/5000]: Train loss: 6.3482, Valid loss: 1.0596
Epoch [416/5000]: Train loss: 6.5441, Valid loss: 1.1460
Epoch [417/5000]: Train loss: 6.1788, Valid loss: 1.0566
Epoch [418/5000]: Train loss: 6.5531, Valid loss: 0.9629
Epoch [419/5000]: Train loss: 6.6753, Valid loss: 0.8957
Epoch [420/5000]: Train loss: 6.1529, Valid loss: 0.8482
Epoch [421/5000]: Train loss: 6.9574, Valid loss: 0.8405
Epoch [422/5000]: Train loss: 6.6482, Valid loss: 0.9734
Epoch [423/5000]: Train loss: 6.7076, Valid loss: 1.0870
Epoch [424/5000]: Train loss: 6.7995, Valid loss: 0.9989
Epoch [425/5000]: Train loss: 6

Epoch [553/5000]: Train loss: 5.8615, Valid loss: 1.1658
Epoch [554/5000]: Train loss: 6.3184, Valid loss: 1.0646
Epoch [555/5000]: Train loss: 5.7351, Valid loss: 0.9713
Epoch [556/5000]: Train loss: 6.1290, Valid loss: 1.0266
Epoch [557/5000]: Train loss: 6.4281, Valid loss: 1.1139
Epoch [558/5000]: Train loss: 6.5640, Valid loss: 0.9795
Epoch [559/5000]: Train loss: 6.2447, Valid loss: 1.1974
Epoch [560/5000]: Train loss: 6.5297, Valid loss: 1.3013
Epoch [561/5000]: Train loss: 6.6449, Valid loss: 1.0220
Epoch [562/5000]: Train loss: 6.7247, Valid loss: 0.9816
Epoch [563/5000]: Train loss: 6.1911, Valid loss: 1.1680
Epoch [564/5000]: Train loss: 6.5432, Valid loss: 1.0636
Epoch [565/5000]: Train loss: 6.6845, Valid loss: 1.0783
Epoch [566/5000]: Train loss: 6.6920, Valid loss: 1.1909
Epoch [567/5000]: Train loss: 6.4235, Valid loss: 1.2957
Epoch [568/5000]: Train loss: 6.5581, Valid loss: 1.2772
Epoch [569/5000]: Train loss: 6.3704, Valid loss: 1.2525
Epoch [570/5000]: Train loss: 6

...

Epoch [3767/5000]: Train loss: 2.6869, Valid loss: 0.9749
Epoch [3768/5000]: Train loss: 2.7502, Valid loss: 1.1293
Epoch [3769/5000]: Train loss: 2.7559, Valid loss: 1.0337
Epoch [3770/5000]: Train loss: 2.6737, Valid loss: 1.1482
Epoch [3771/5000]: Train loss: 2.7844, Valid loss: 1.0107
Epoch [3772/5000]: Train loss: 3.1726, Valid loss: 1.1458
Epoch [3773/5000]: Train loss: 2.7470, Valid loss: 1.0308
Epoch [3774/5000]: Train loss: 2.8688, Valid loss: 0.9019
Epoch [3775/5000]: Train loss: 2.8429, Valid loss: 0.8817
Epoch [3776/5000]: Train loss: 2.6228, Valid loss: 1.0199
Epoch [3777/5000]: Train loss: 2.7902, Valid loss: 0.9950
Epoch [3778/5000]: Train loss: 2.6823, Valid loss: 1.1459
Epoch [3779/5000]: Train loss: 2.5841, Valid loss: 1.1563
Epoch [3780/5000]: Train loss: 3.1064, Valid loss: 0.8807
Epoch [3781/5000]: Train loss: 2.7690, Valid loss: 0.8899
Epoch [3782/5000]: Train loss: 2.8869, Valid loss: 0.8294
Epoch [3783/5000]: Train loss: 3.0002, Valid loss: 1.0061
Epoch [3784/50

Epoch [3910/5000]: Train loss: 2.7422, Valid loss: 1.0105
Epoch [3911/5000]: Train loss: 2.6835, Valid loss: 0.9911
Epoch [3912/5000]: Train loss: 2.8426, Valid loss: 1.0494
Epoch [3913/5000]: Train loss: 2.8268, Valid loss: 0.9297
Epoch [3914/5000]: Train loss: 2.8703, Valid loss: 1.0946
Epoch [3915/5000]: Train loss: 2.7878, Valid loss: 0.8908
Epoch [3916/5000]: Train loss: 2.7672, Valid loss: 1.0073
Epoch [3917/5000]: Train loss: 2.8403, Valid loss: 1.0310
Epoch [3918/5000]: Train loss: 2.9520, Valid loss: 0.8347
Epoch [3919/5000]: Train loss: 2.5815, Valid loss: 0.8809
Epoch [3920/5000]: Train loss: 2.8385, Valid loss: 1.0401
Epoch [3921/5000]: Train loss: 2.5479, Valid loss: 0.9818
Epoch [3922/5000]: Train loss: 2.6147, Valid loss: 0.9125
Epoch [3923/5000]: Train loss: 2.5961, Valid loss: 0.9589
Epoch [3924/5000]: Train loss: 3.3009, Valid loss: 1.0127
Epoch [3925/5000]: Train loss: 2.7778, Valid loss: 1.0160
Epoch [3926/5000]: Train loss: 2.9021, Valid loss: 0.9980
Epoch [3927/50

Epoch [4057/5000]: Train loss: 2.6379, Valid loss: 0.9933
Epoch [4058/5000]: Train loss: 2.6558, Valid loss: 0.9233
Epoch [4059/5000]: Train loss: 2.9595, Valid loss: 0.9150
Epoch [4060/5000]: Train loss: 3.2199, Valid loss: 0.9038
Epoch [4061/5000]: Train loss: 2.4739, Valid loss: 0.9422
Epoch [4062/5000]: Train loss: 2.9783, Valid loss: 0.9979
Epoch [4063/5000]: Train loss: 3.1834, Valid loss: 1.0196
Epoch [4064/5000]: Train loss: 2.6339, Valid loss: 0.9709
Epoch [4065/5000]: Train loss: 2.7413, Valid loss: 1.2543
Epoch [4066/5000]: Train loss: 2.6095, Valid loss: 0.9364
Epoch [4067/5000]: Train loss: 2.5060, Valid loss: 0.9780
Epoch [4068/5000]: Train loss: 2.6953, Valid loss: 0.9303
Epoch [4069/5000]: Train loss: 2.3713, Valid loss: 0.8821
Epoch [4070/5000]: Train loss: 2.5435, Valid loss: 0.9967
Epoch [4071/5000]: Train loss: 2.6766, Valid loss: 0.9753
Epoch [4072/5000]: Train loss: 2.7147, Valid loss: 0.9411
Epoch [4073/5000]: Train loss: 2.8686, Valid loss: 0.9124
Epoch [4074/50

Epoch [4202/5000]: Train loss: 2.6760, Valid loss: 0.9349
Epoch [4203/5000]: Train loss: 2.9775, Valid loss: 0.9750
Epoch [4204/5000]: Train loss: 2.4501, Valid loss: 0.8872
Epoch [4205/5000]: Train loss: 3.0234, Valid loss: 1.0187
Epoch [4206/5000]: Train loss: 2.7652, Valid loss: 1.0454
Epoch [4207/5000]: Train loss: 2.4459, Valid loss: 1.1876
Epoch [4208/5000]: Train loss: 2.6205, Valid loss: 0.9825
Epoch [4209/5000]: Train loss: 2.7363, Valid loss: 0.9526
Epoch [4210/5000]: Train loss: 2.8516, Valid loss: 1.0528
Epoch [4211/5000]: Train loss: 3.2320, Valid loss: 1.0620
Epoch [4212/5000]: Train loss: 2.5769, Valid loss: 1.1323
Epoch [4213/5000]: Train loss: 2.6805, Valid loss: 1.1103
Epoch [4214/5000]: Train loss: 2.9384, Valid loss: 1.0176
Epoch [4215/5000]: Train loss: 2.6442, Valid loss: 1.1836
Epoch [4216/5000]: Train loss: 2.6229, Valid loss: 1.0596
Epoch [4217/5000]: Train loss: 2.7967, Valid loss: 1.1279
Epoch [4218/5000]: Train loss: 2.7944, Valid loss: 0.9237
Epoch [4219/50

Epoch [4347/5000]: Train loss: 2.7501, Valid loss: 1.0554
Epoch [4348/5000]: Train loss: 3.0189, Valid loss: 1.1749
Epoch [4349/5000]: Train loss: 2.7573, Valid loss: 1.0252
Epoch [4350/5000]: Train loss: 2.7530, Valid loss: 1.0949
Epoch [4351/5000]: Train loss: 2.7137, Valid loss: 1.2762
Epoch [4352/5000]: Train loss: 2.5963, Valid loss: 0.7917
Epoch [4353/5000]: Train loss: 2.8123, Valid loss: 0.9729
Epoch [4354/5000]: Train loss: 2.6150, Valid loss: 1.0207
Epoch [4355/5000]: Train loss: 2.6869, Valid loss: 1.2168
Epoch [4356/5000]: Train loss: 2.9540, Valid loss: 0.7998
Epoch [4357/5000]: Train loss: 2.7859, Valid loss: 0.8056
Epoch [4358/5000]: Train loss: 2.5629, Valid loss: 0.8695
Epoch [4359/5000]: Train loss: 2.3370, Valid loss: 0.9351
Epoch [4360/5000]: Train loss: 3.0088, Valid loss: 0.8927
Epoch [4361/5000]: Train loss: 2.6566, Valid loss: 1.1157
Epoch [4362/5000]: Train loss: 3.2277, Valid loss: 1.0304
Epoch [4363/5000]: Train loss: 3.0619, Valid loss: 0.9612
Epoch [4364/50

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [96]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 378.26it/s]


# Download

Run this block to download the `pred.csv` by clicking.

In [63]:
from IPython.display import FileLink
FileLink(r'pred.csv')

/home/users/guanyitong/gyt-dev/gyt_test/algorithm/HUNG-YI-LEE/23_hw1_regression/pred.csv

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)